In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist = tfds.load('mnist')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [2]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
EPOCHS = 20

step_per_epoch = np.ceil(60000 / BATCH_SIZE)

In [3]:
def preprocess_image(item):
    image = item['image']
    label = item['label']
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.reshape(image, (-1,))
    
    return {'image-pixels': image}, label[..., tf.newaxis]

- Preprocessing

In [4]:
def input_images(split:str, shuffle=False, repeat=False):
    dataset = tfds.load(name='mnist')
    data_split = dataset[split]
    dataset = data_split.map(preprocess_image)
    
    if shuffle:
        dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    if repeat:
        dataset = dataset.repeat()
        
    return dataset

In [5]:
#Feature columns.
image_feature_column = tf.feature_column.numeric_column(key='image-pixels', shape=(28*28))

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


 - Creating and instatiating the estimator.

In [ ]:
dnn_classifier = tf.estimator.DNNClassifier(
    feature_columns=[image_feature_column],
    hidden_units=[32, 16],
    n_classes=10,
    model_dir='../models/mnist-dnn/'
)

Instructions for updating:
Use tf.keras instead.


Instructions for updating:
Use tf.keras instead.


Instructions for updating:
Use tf.keras instead.


Instructions for updating:
Use tf.keras instead.


Instructions for updating:
Use tf.keras instead.


...

Instructions for updating:
Use tf.keras instead.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '../models/mnist-dnn/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '../models/mnist-dnn/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


- Training and Evaluation.

In [ ]:
dnn_classifier.train(
    input_fn=input_images('train', shuffle=True, repeat=True),
    steps=EPOCHS*step_per_epoch
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

ConnectionError: HTTPSConnectionPool(host='storage.googleapis.com', port=443): Max retries exceeded with url: /cvdf-datasets/mnist/t10k-images-idx3-ubyte.gz (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x752f1825ee60>: Failed to resolve 'storage.googleapis.com' ([Errno -3] Temporary failure in name resolution)"))

In [ ]:
eval_result = dnn_classifier.evaluate(
    input_fn=input_images('test')
    )

In [ ]:
print(eval_result)

### Creating a custom Estimator from an existing Keras model.

In [6]:
tf.random.set_seed(1)

x = np.random.uniform(low=-1, high=1, size=(200, 2))
y = np.ones(shape=len(x))
y[x[:, 0] * x[:, 1] < 0] = 0

x_train = x[100:, :]
y_train = y[100:]
x_valid = x[:100, :]
y_valid = y[:100]

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,), name='input_features'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

- Define input functions. 

In [14]:
def train_input_func(x_train, y_train, batch_size=8):
    dataset = tf.data.Dataset.from_tensor_slices(
        ({'input_features': x_train}, y_train.reshape(-1, 1))
    )
    
    #Shuffle,repeat, batch
    return dataset.shuffle(100).repeat().batch(batch_size)

def eval_input_func(x_test, y_test=None, batch_size=8):
    if y_test is None:
        dataset = tf.data.Dataset.from_tensor_slices(
        ({'input_features': x_test})
        )
    else:
        dataset = tf.data.Dataset.from_tensor_slices(
            ({'input_features': x_test}, y_test.reshape(-1, 1))
        )
    
    return dataset.batch(batch_size)

In [15]:
#Feature columns.
features = tf.feature_column.numeric_column(
    key='input_features', shape=(2,)
)

- Compile and make to an estimator.

In [10]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])


In [11]:
unoEstimator = tf.keras.estimator.model_to_estimator(
    keras_model=model,
    model_dir='../models/keras_estimator_XOR/'
)

Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.


Instructions for updating:
Use tf.keras instead.


Instructions for updating:
Use tf.keras instead.


INFO:tensorflow:Using config: {'_model_dir': '../models/keras_estimator_XOR/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '../models/keras_estimator_XOR/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Instructions for updating:
Use tf.keras instead.


Instructions for updating:
Use tf.keras instead.


 - Use the estimator.

In [16]:
num_epochs = 200
batch_size = 2
steps_per_epoch = np.ceil(len(x_train) / batch_size)

unoEstimator.train(
    input_fn=lambda: train_input_func(x_train, y_train, batch_size),
    steps=num_epochs*steps_per_epoch
)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Saving checkpoints for 20000 into ../models/keras_estimator_XOR/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 20000...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 20000...


INFO:tensorflow:Loss for final step: 0.0072699115.


INFO:tensorflow:Loss for final step: 0.0072699115.


In [17]:
unoEstimator.evaluate(
    input_fn=lambda: eval_input_func(x_valid, y_valid, batch_size)
)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2024-06-04T18:03:06


...

INFO:tensorflow:Saving dict for global step 20000: binary_accuracy = 0.94, global_step = 20000, loss = 0.25957197


INFO:tensorflow:Saving dict for global step 20000: binary_accuracy = 0.94, global_step = 20000, loss = 0.25957197


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: ../models/keras_estimator_XOR/model.ckpt-20000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: ../models/keras_estimator_XOR/model.ckpt-20000


{'binary_accuracy': 0.94, 'loss': 0.25957197, 'global_step': 20000}